In [9]:
import sys
sys.path.append("../")

import dataInterpreter as dt
import enrichmentAnalysis as ea
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
pd.options.mode.chained_assignment = None  # default='warn'

import xgboost as xgb
from sklearn.metrics import mean_squared_error

path = "C:\\Users\\Pedro\\Documents\\BicPAMS\\bicpams_5.1\\data\\latecovid\\"

In [10]:
filtered_data_01 = pd.read_csv(path + 'data-p01.csv', index_col = 0, sep = '\t').T
filtered_data_05 = pd.read_csv(path + 'data-p05.csv', index_col = 0, sep = '\t').T

In [11]:
classes = {'NHBE': {}, 'A549': {}, 'Calu3': {}, 'Biopsy': {}}
i = 0
for c in filtered_data_01.index:
    info = dt.get_info_from_name(c)
    if info['Condition'] not in classes[info['Cell Type']]:
        classes[info['Cell Type']][info['Condition']] = i
        i += 1
        
y = []
for c in filtered_data_01.index:
    info = dt.get_info_from_name(c)
    y.append(classes[info['Cell Type']][info['Condition']])
    
y_names = [' '] * 13
for c_type in classes:
    for cond in classes[c_type]:
        y_names[classes[c_type][cond]] = c_type + ' ' + cond


In [12]:
parameters = {
    'seed': 42, 
    'use_label_encoder': False,
    'booster': 'gbtree',
    'eta': 0.3,
    'gamma': 0,
    'alpha': 0,
    'n_estimators': 400,
    'eval_metric': 'mlogloss'
}

stats = dt.apply_loocv(filtered_data_01.values, np.array(y), xgb.XGBClassifier(**parameters))
stats

{'accuracy': 0.8333333333333334,
 'importances': array([1.51567477e-02, 1.87414878e-03, 2.25780616e-04, 1.53541223e-02,
        1.07414615e-03, 8.30417431e-03, 1.03555782e-03, 9.25054644e-03,
        2.27620066e-05, 2.35013421e-03, 3.03968835e-05, 3.90527527e-03,
        1.69028337e-02, 1.14295912e-03, 7.40122368e-02, 4.49351856e-04,
        2.07753115e-04, 7.52164859e-04, 2.17970493e-03, 1.02165082e-03,
        2.08289449e-04, 7.29739627e-04, 1.95741645e-02, 2.88464549e-03,
        0.00000000e+00, 1.92097387e-03, 1.28293403e-06, 1.17236435e-03,
        3.48453163e-04, 2.28825638e-04, 6.84369476e-03, 0.00000000e+00,
        2.59701949e-04, 2.11082198e-03, 9.35280095e-03, 0.00000000e+00,
        1.12821759e-05, 6.25204036e-04, 1.77100986e-04, 3.07144733e-03,
        1.88006312e-04, 1.69814823e-04, 9.93971402e-04, 4.14471061e-02,
        0.00000000e+00, 4.96382590e-06, 8.79292670e-04, 0.00000000e+00,
        2.55225588e-05, 9.36857860e-04, 0.00000000e+00, 1.93885027e-04,
        1.729243

In [13]:
clf = xgb.XGBClassifier(**parameters).fit(filtered_data_01, y)

list(sorted(zip(clf.feature_importances_, filtered_data_01.columns), reverse = True))

[(0.17600389, 'IL1A'),
 (0.08085336, 'NKAIN1'),
 (0.040582884, 'FSIP1'),
 (0.033085812, 'SAMD9L'),
 (0.025526734, 'EBI3'),
 (0.024465673, 'IFI16'),
 (0.024256032, 'CXCL1'),
 (0.023557836, 'NRTN'),
 (0.02271954, 'UBE2L6'),
 (0.020057864, 'FHL5'),
 (0.018765423, 'DCLK1'),
 (0.016079819, 'PHF11'),
 (0.0154207535, 'THEMIS2'),
 (0.015012367, 'GBP2'),
 (0.014706367, 'GFPT2'),
 (0.014257988, 'HCN4'),
 (0.012703476, 'HERC5'),
 (0.012473829, 'TNFRSF9'),
 (0.012232057, 'PDZK1IP1'),
 (0.011772433, 'SSH1'),
 (0.011768914, 'CFB'),
 (0.0113923615, 'CASP4'),
 (0.011376115, 'CLDN16'),
 (0.011316258, 'MOCOS'),
 (0.011273014, 'NMNAT1'),
 (0.011028717, 'SPATA21'),
 (0.010886526, 'KRT35'),
 (0.010886447, 'GCH1'),
 (0.010847612, 'KYNU'),
 (0.010838783, 'ICAM1'),
 (0.010381575, 'IFIT3'),
 (0.01029181, 'TLR2'),
 (0.010231779, 'FBLIM1'),
 (0.010216653, 'FHDC1'),
 (0.0100672785, 'CCBE1'),
 (0.0100054415, 'STX12'),
 (0.009745724, 'MT2A'),
 (0.009466907, 'SAA1'),
 (0.009324187, 'ZBTB25'),
 (0.00850817, 'CX3CL1')

In [14]:
selected_genes = [x[1] for x in list(sorted(zip(clf.feature_importances_, filtered_data_01.columns), reverse = True)) if x[0] > 0]
selected_genes

['IL1A',
 'NKAIN1',
 'FSIP1',
 'SAMD9L',
 'EBI3',
 'IFI16',
 'CXCL1',
 'NRTN',
 'UBE2L6',
 'FHL5',
 'DCLK1',
 'PHF11',
 'THEMIS2',
 'GBP2',
 'GFPT2',
 'HCN4',
 'HERC5',
 'TNFRSF9',
 'PDZK1IP1',
 'SSH1',
 'CFB',
 'CASP4',
 'CLDN16',
 'MOCOS',
 'NMNAT1',
 'SPATA21',
 'KRT35',
 'GCH1',
 'KYNU',
 'ICAM1',
 'IFIT3',
 'TLR2',
 'FBLIM1',
 'FHDC1',
 'CCBE1',
 'STX12',
 'MT2A',
 'SAA1',
 'ZBTB25',
 'CX3CL1',
 'C22orf24',
 'RGS17',
 'TRANK1',
 'GRID1',
 'PTPRH',
 'IFITM1',
 'IFI44',
 'SAMD9',
 'FAM49A',
 'SYTL2',
 'SLC6A4',
 'ADD2',
 'PCK2',
 'CXCL5',
 'ANKRD26P1',
 'CATSPER2P1',
 'MCAM',
 'IRF2BPL',
 'IGFBP1',
 'PPP1R16B',
 'ARHGEF4',
 'ZNF862',
 'LOC254896',
 'PITPNA-AS1',
 'MOBP',
 'IRF7',
 'IFIT2',
 'CCL20',
 'GPSM3',
 'MYPN',
 'HBEGF',
 'TRABD2A',
 'ADORA2A',
 'ZNF239',
 'CCL5',
 'PLIN2',
 'JUN',
 'BMPER',
 'TFF1',
 'STC2',
 'FOXD1',
 'RPL36A',
 'MAP1B',
 'NMI',
 'PARP10',
 'C3AR1',
 'MB',
 'CCL2',
 'XAF1',
 'HSF2BP',
 'REEP2',
 'PI3',
 'DAW1',
 'TCTE3']

In [15]:
results = ea.getEnrichment(selected_genes, 'GO_Biological_Process_2021')['GO_Biological_Process_2021']
results

Order of returned results is: Rank, Term name, P-value, Z-score, Combined score, Overlapping genes, Adjusted p-value, Old p-value, Old adjusted p-value


[[1,
  'response to interferon-gamma (GO:0034341)',
  1.3336127778476142e-10,
  29.579950289975145,
  672.5877056730069,
  ['IFITM1',
   'GCH1',
   'CCL20',
   'KYNU',
   'CCL5',
   'CCL2',
   'GBP2',
   'CX3CL1',
   'TLR2'],
  1.19625066172931e-07,
  0,
  0],
 [2,
  'cytokine-mediated signaling pathway (GO:0019221)',
  5.636790875266018e-10,
  7.581696779261587,
  161.46387855314202,
  ['IFITM1',
   'CCL20',
   'TNFRSF9',
   'EBI3',
   'CXCL1',
   'CX3CL1',
   'CXCL5',
   'IFIT3',
   'IFIT2',
   'ICAM1',
   'IL1A',
   'MT2A',
   'CCL5',
   'IRF7',
   'SAA1',
   'CCL2',
   'GBP2',
   'XAF1'],
  2.528100707556809e-07,
  0,
  0],
 [3,
  'cellular response to interferon-gamma (GO:0071346)',
  5.5515596247830145e-09,
  18.71281512605042,
  355.7154008147536,
  ['MT2A', 'CCL20', 'CCL5', 'IRF7', 'CCL2', 'GBP2', 'CX3CL1', 'TLR2', 'ICAM1'],
  1.6599163278101214e-06,
  0,
  0],
 [4,
  'neutrophil chemotaxis (GO:0030593)',
  3.8032671024636885e-08,
  25.34227330779055,
  432.9681850211753,
  ['C

In [16]:
import json

with open('results_xGBoost.json', 'w') as file:
     file.write(json.dumps(results)) # use `json.loads` to do the reverse

In [17]:
import json

with open('results_xGBoost.json') as file:
    results = json.load(file)

In [18]:
dataset = {'p-value': [], 'Score': []}
index = []

for term in results:
    index += [term[1]]
    dataset['p-value'] += [term[6]]
    dataset['Score'] += [term[4]]
enrichment_dataset = pd.DataFrame(dataset, index = index)

In [21]:
len(enrichment_dataset[enrichment_dataset['p-value'] < 0.001].index)

15

In [ ]:
pd.set_option("display.max_rows", None)
selection = enrichment_dataset[enrichment_dataset['p-value'] < 0.01].sort_values('Score', ascending = False).head(25)

selection['p-value'] = selection['p-value'].map(lambda x: '%.2E' % x)
selection['Score'] = selection['Score'].map(lambda x: '%.2f' % x)

#selection.to_csv('xGBoost_table.csv')
selection

### Two class comparison

##### NHBE

In [ ]:
cols_nhbe_healthy = dt.get_columns('NHBE', 'healthy')
cols_nhbe_cov2 = dt.get_columns('NHBE', 'sars-cov2')

labels_nhbe = [0] * len(cols_nhbe_healthy) + [1] * len(cols_nhbe_cov2)

data_nhbe = dt.get_data('NHBE', 'healthy', 'sars-cov2')

filtered_data_NHBE = dt.get_p_values('mannwhitneyu', data_nhbe, cols_nhbe_healthy, cols_nhbe_cov2)
filtered_data_NHBE.drop(['p-value'], axis = 1, inplace = True)

len(filtered_data_NHBE.index)

In [ ]:
parameters = {
    'seed': 42, 
    'use_label_encoder': False,
    'booster': 'gbtree',
    'eta': 0.3,
    'gamma': 0,
    'alpha': 0,
    'n_estimators': 200,
    'eval_metric': 'mlogloss',
}

stats_NHBE = dt.apply_loocv(filtered_data_NHBE.T.values, np.array(labels_nhbe), xgb.XGBClassifier(**parameters))
stats_NHBE

In [ ]:
nhbe_clf = xgb.XGBClassifier(**parameters).fit(filtered_data_NHBE.T, labels_nhbe)

list(sorted(zip(nhbe_clf.feature_importances_, filtered_data_NHBE.T.columns), reverse = True))

In [ ]:
selected_genes_nhbe = [x[1] for x in list(sorted(zip(nhbe_clf.feature_importances_, filtered_data_NHBE.T.columns), reverse = True)) if x[0] > 0]
selected_genes_nhbe

In [ ]:
results_nhbe = ea.getEnrichment(selected_genes_nhbe, 'GO_Biological_Process_2021')['GO_Biological_Process_2021']
results_nhbe

In [ ]:
import json

with open('results_xGBoost_NHBE.json', 'w') as file:
     #file.write(json.dumps(results_nhbe)) # use `json.loads` to do the reverse

In [22]:
import json

with open('results_xGBoost_NHBE.json') as file:
    results_nhbe = json.load(file)

In [23]:
dataset = {'p-value': [], 'Score': [], 'Value': []}
index_nhbe = []

cols_nhbe_healthy = dt.get_columns('NHBE', 'healthy')
cols_nhbe_cov2 = dt.get_columns('NHBE', 'sars-cov2')

data_nhbe = dt.get_data('NHBE', 'healthy', 'sars-cov2')

for term in results_nhbe:
    index_nhbe += [term[1]]
    dataset['p-value'] += [term[6]]
    dataset['Score'] += [term[4]]
    
    genes = term[5]
    avg_sub = np.mean(data_nhbe.loc[genes, cols_nhbe_healthy].values, axis = 1) - np.mean(data_nhbe.loc[genes, cols_nhbe_cov2].values, axis = 1)

    downs = 0
    ups = 0

    for e in avg_sub:
        if e > 0:
            downs += 1
        elif e < 0:
            ups += 1

    dataset['Value'] += ['%d up, %d down' % (ups, downs)]
    
enrichment_nhbe_dataset = pd.DataFrame(dataset, index = index_nhbe)

In [26]:
len(enrichment_nhbe_dataset[enrichment_nhbe_dataset['p-value'] < 0.01].index)

0

In [ ]:
pd.set_option("display.max_rows", None)
selection = enrichment_nhbe_dataset[enrichment_nhbe_dataset['p-value'] < 0.05].sort_values('Score', ascending = False)#.head(25)[['p-value', 'Score']]

selection['p-value'] = selection['p-value'].map(lambda x: '%.2E' % x)
selection['Score'] = selection['Score'].map(lambda x: '%.2f' % x)

#selection.to_csv('NHBE_xGBoost_table.csv')
selection

##### A549

In [ ]:
cols_healthy_A549 = dt.get_columns('A549', 'healthy')
cols_cov2_A549 = dt.get_columns('A549', 'sars-cov2')

labels_a549 = [0] * len(cols_healthy_A549) + [1] * len(cols_cov2_A549)

data_a549 = dt.get_data('A549', 'healthy', 'sars-cov2')

filtered_data_a549 = dt.get_p_values('mannwhitneyu', data_a549, cols_healthy_A549, cols_cov2_A549, limit = 0.01)
filtered_data_a549.drop(['p-value'], axis = 1, inplace = True)

len(filtered_data_a549)

In [ ]:
parameters = {
    'seed': 42, 
    'use_label_encoder': False,
    'booster': 'gbtree',
    'learning_rate': 0.3,
    'gamma': 0,
    'alpha': 0,
    'n_estimators': 300,
    'eval_metric': 'mlogloss',
}

stats_a549 = dt.apply_loocv(filtered_data_a549.T.values, np.array(labels_a549), xgb.XGBClassifier(**parameters))
stats_a549

In [ ]:
a549_clf = xgb.XGBClassifier(**parameters).fit(filtered_data_a549.T, labels_a549)

list(sorted(zip(a549_clf.feature_importances_, filtered_data_a549.T.columns), reverse = True))

In [ ]:
selected_genes_a549 = [x[1] for x in list(sorted(zip(a549_clf.feature_importances_, filtered_data_a549.T.columns), reverse = True)) if x[0] > 0]
selected_genes_a549

In [ ]:
results_a549 = ea.getEnrichment(selected_genes_a549, 'GO_Biological_Process_2021')['GO_Biological_Process_2021']
results_a549

In [ ]:
import json

with open('results_xGBoost_A549.json', 'w') as file:
     #file.write(json.dumps(results_a549)) # use `json.loads` to do the reverse

In [ ]:
import json

with open('results_xGBoost_A549.json') as file:
    results_a549 = json.load(file)

In [ ]:
dataset = {'p-value': [], 'Score': [], 'Value': []}
index_a549 = []

cols_healthy_A549 = dt.get_columns('A549', 'healthy')
cols_cov2_A549 = dt.get_columns('A549', 'sars-cov2')

data_a549 = dt.get_data('A549', 'healthy', 'sars-cov2')

for term in results_a549:
    index_a549 += [term[1]]
    dataset['p-value'] += [term[6]]
    dataset['Score'] += [term[4]]
    
    genes = term[5]
    avg_sub = np.mean(data_a549.loc[genes, cols_healthy_A549].values, axis = 1) - np.mean(data_a549.loc[genes, cols_cov2_A549].values, axis = 1)

    downs = 0
    ups = 0

    for e in avg_sub:
        if e > 0:
            downs += 1
        elif e < 0:
            ups += 1

    dataset['Value'] += ['%d up, %d down' % (ups, downs)]
enrichment_a549_dataset = pd.DataFrame(dataset, index = index_a549)

In [ ]:
len(enrichment_a549_dataset[enrichment_a549_dataset['p-value'] < 0.01].index)

In [ ]:
pd.set_option("display.max_rows", None)
selection = enrichment_a549_dataset[enrichment_a549_dataset['p-value'] < 0.05].sort_values('Score', ascending = False).head(25)[['p-value', 'Score']]

selection['p-value'] = selection['p-value'].map(lambda x: '%.2E' % x)
selection['Score'] = selection['Score'].map(lambda x: '%.2f' % x)

#selection.to_csv('A549_xGBoost_table.csv')
selection

##### Calu3

In [ ]:
cols_healthy_Calu3 = dt.get_columns('Calu3', 'healthy')
cols_cov2_Calu3 = dt.get_columns('Calu3', 'sars-cov2')

labels_calu3 = [0] * len(cols_healthy_Calu3) + [1] * len(cols_cov2_Calu3)

data_calu3 = dt.get_data('Calu3', 'healthy', 'sars-cov2')

filtered_data_calu3 = dt.get_p_values('mannwhitneyu', data_calu3, cols_healthy_Calu3, cols_cov2_Calu3)
filtered_data_calu3.drop(['p-value'], axis = 1, inplace = True)

len(filtered_data_calu3.index)

In [ ]:
parameters = {
    'seed': 42, 
    'use_label_encoder': False,
    'booster': 'gbtree',
    'learning_rate': 0.3,
    'gamma': 0,
    'alpha': 0,
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
}

stats_calu3 = dt.apply_loocv(filtered_data_calu3.T.values, np.array(labels_calu3), xgb.XGBClassifier(**parameters))
stats_calu3

In [ ]:
calu3_clf = xgb.XGBClassifier(**parameters).fit(filtered_data_calu3.T, labels_calu3)

list(sorted(zip(calu3_clf.feature_importances_, filtered_data_calu3.T.columns), reverse = True))

In [ ]:
xgb.XGBClassifier()

In [ ]:
selected_genes_a549 = [x[1] for x in list(sorted(zip(a549_clf.feature_importances_, filtered_data_a549.T.columns), reverse = True)) if x[0] > 0]
selected_genes_a549

In [ ]:
results_calu3 = ea.getEnrichment(calu3_features[:92], 'GO_Biological_Process_2021')['GO_Biological_Process_2021']
results_calu3

In [ ]:
import json

with open('results_xGBoost_Calu3.json', 'w') as file:
     file.write(json.dumps(results_calu3)) # use `json.loads` to do the reverse

In [ ]:
import json

with open('results_xGBoost_Calu3.json') as file:
    results_calu3 = json.load(file)

In [ ]:
dataset = {'p-value': [], 'Score': [], 'Value': []}
index_calu3 = []

cols_healthy_Calu3 = dt.get_columns('Calu3', 'healthy')
cols_cov2_Calu3 = dt.get_columns('Calu3', 'sars-cov2')

data_calu3 = dt.get_data('Calu3', 'healthy', 'sars-cov2')

for term in results_calu3:
    index_calu3 += [term[1]]
    dataset['p-value'] += [term[6]]
    dataset['Score'] += [term[4]]
    
    genes = term[5]
    avg_sub = np.mean(data_calu3.loc[genes, cols_healthy_Calu3].values, axis = 1) - np.mean(data_calu3.loc[genes, cols_cov2_Calu3].values, axis = 1)

    downs = 0
    ups = 0

    for e in avg_sub:
        if e > 0:
            downs += 1
        elif e < 0:
            ups += 1

    dataset['Value'] += ['%d up, %d down' % (ups, downs)]
    
enrichment_calu3_dataset = pd.DataFrame(dataset, index = index_calu3)

In [ ]:
pd.set_option("display.max_rows", None)
selection = enrichment_calu3_dataset[enrichment_calu3_dataset['p-value'] < 0.05].sort_values('Score', ascending = False).head(25)[['p-value', 'Score']]

selection['p-value'] = selection['p-value'].map(lambda x: '%.2E' % x)
selection['Score'] = selection['Score'].map(lambda x: '%.2f' % x)

#selection.to_csv('Calu3_xGBoost_table.csv')
selection